In [1]:
import psycopg2 as ps
import pandas as pd

In [2]:
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)
    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
        return conn

In [3]:
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS videos (
                    video_id VARCHAR(255) PRIMARY KEY,
                    video_title TEXT NOT NULL,
                    upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
                    view_count BIGINT NOT NULL,
                    like_count BIGINT NOT NULL,
                    dislike_count BIGINT NOT NULL,
                    comment_count BIGINT NOT NULL
            )""")
    curr.execute(create_table_command)

In [4]:
def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count, dislike_count, comment_count):
    insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date,
                        view_count, like_count, dislike_count,comment_count)
    VALUES(%s,%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (video_id, video_title, upload_date, view_count, like_count, dislike_count, comment_count)
    curr.execute(insert_into_videos, row_to_insert)

In [5]:
def update_row(curr, video_id, video_title, view_count, like_count, dislike_count, comment_count):
    query = ("""UPDATE videos
            SET video_title = %s,
                view_count = %s,
                like_count = %s,
                dislike_count = %s,
                comment_count = %s
            WHERE video_id = %s;""")
    vars_to_update = (video_title, view_count, like_count, dislike_count, comment_count, video_id)
    curr.execute(query, vars_to_update)

In [6]:
def check_if_video_exists(curr, video_id): 
    query = ("""SELECT video_id FROM VIDEOS WHERE video_id = %s""")
    curr.execute(query, (video_id,))
    return curr.fetchone() is not None

In [7]:
# def truncate_table(curr):
#     truncate_table = ("""TRUNCATE TABLE videos""")
#     curr.execute(truncate_table)

In [8]:
def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count']
                          , row['like_count'], row['dislike_count'], row['comment_count'])

In [9]:
def update_db(curr,df):
    tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count',
                                   'like_count', 'dislike_count', 'comment_count'])
    for i, row in df.iterrows():
        if check_if_video_exists(curr, row['video_id']): # If video already exists then we will update
            update_row(curr,row['video_id'],row['video_title'],row['view_count'],row['like_count'],row['dislike_count'],row['comment_count'])
        else: # The video doesn't exists so we will add it to a temp df and append it using append_from_df_to_db
            tmp_df = tmp_df.append(row)
    return tmp_df

In [10]:
df = pd.read_csv('data.csv', index_col=0)
df['upload_date'] = df['upload_date'] = pd.to_datetime(df['upload_date'])
df.head()

,video_id,video_title,upload_date,view_count,like_count,dislike_count,comment_count
0,kLkmGUTwG-s,Comment | Java for Beginners | Bangla Tutorial...,2021-01-05,67,4,0,0
1,baGEPw5XMog,How To Run A Java Program | Java for Beginners...,2020-10-18,155,6,0,1
2,47y_BeKHIPs,C++ vs Java | Java for Beginners | Bangla Tuto...,2020-09-28,123,2,0,1
3,sKUZAeyF_QU,Overview of Java | Java for Beginners | Bangla...,2020-09-03,100,7,0,2
4,hl4eq1ToX3U,How To Add A Calendar To A Website | Bangla We...,2020-08-31,70,10,0,4


In [11]:
print(type(df['upload_date'][1]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [12]:
#database credentials
host_name = 'x.rds.amazonaws.com'
dbname = 'dbname'
port = '5432'
username = 'xyz'
password = 'xyz'

#establish a connection to db
conn = connect_to_db(host_name, dbname, port, username, password)
curr = conn.cursor()

Connected!


In [13]:
create_table(curr)

In [14]:
new_vid_df = update_db(curr, df)
conn.commit()

In [15]:
append_from_df_to_db(curr, new_vid_df)
conn.commit()

In [16]:
# inse = """INSERT INTO videos (video_id, video_title, upload_date,
#                         view_count, like_count, dislike_count,comment_count)
#     VALUES('1212sdssfsad','nach meri rani','2022-12-30',122,10,0,2);"""
# curr.execute(inse)

In [17]:
#view data in db table
curr.execute("SELECT * FROM VIDEOS")
print(curr.fetchall())

[('kLkmGUTwG-s', 'Comment | Java for Beginners | Bangla Tutorial | #5', datetime.date(2021, 1, 5), 67, 4, 0, 0), ('baGEPw5XMog', 'How To Run A Java Program | Java for Beginners | Bangla Tutorial | #4', datetime.date(2020, 10, 18), 155, 6, 0, 1), ('47y_BeKHIPs', 'C++ vs Java | Java for Beginners | Bangla Tutorial | #3', datetime.date(2020, 9, 28), 123, 2, 0, 1), ('sKUZAeyF_QU', 'Overview of Java | Java for Beginners | Bangla Tutorial | #2', datetime.date(2020, 9, 3), 100, 7, 0, 2), ('hl4eq1ToX3U', 'How To Add A Calendar To A Website | Bangla Web Development Tricks | CS Bangla', datetime.date(2020, 8, 31), 70, 10, 0, 4), ('g_Gei7Ss2Ow', 'Top 10 Programming Languages in 2020 | CS Bangla', datetime.date(2020, 8, 27), 296, 16, 0, 5), ('3c0h8hvDsEw', 'Introduction | Java for Beginners | Bangla Tutorial | #1', datetime.date(2020, 8, 24), 249, 12, 0, 10), ('Koz-r8a1lXQ', 'C++ Class and Object | Bangla Tutorial | Part-18', datetime.date(2020, 8, 17), 613, 21, 0, 3), ('zt6G2Cqxrzs', 'C++ Functio

In [18]:
conn.close()